In [1]:
import com.google.gson.GsonBuilder
import hu.akos.hollo.szabo.math.Permutation
import hu.raven.puppet.logic.statistics.edgeHistogramMatrix
import hu.raven.puppet.logic.statistics.mapStates
import hu.raven.puppet.utility.PermutationTypeAdapter
import hu.raven.puppet.model.state.BacteriophageAlgorithmState
import hu.raven.puppet.model.utility.math.GraphEdge
import java.io.File


# What statistics to create

Usual stuff:
- progress of optimisation
- age distribution
- diversity of population
    - edge recurrence
    - distance based density
    - distance from best based density

Operator specific:
- crossover:
    - distance from parents
    - improvement
- mutation
    - edge change
    - gained distance from parents
  

In [3]:
import hu.akos.hollo.szabo.math.matrix.IntMatrix

val edgeAccuranceMatrices:List<IntMatrix> = TODO()

kotlin.NotImplementedError: An operation is not implemented.

In [4]:
%use kandy
%use dataframe

In [ ]:
val state = edgeAccuranceMatrices[731]

val mappedToEdges = state.indices[0]
    .flatMap { sourceNodeIndex ->
        state.indices[1].map { targetNodeIndex ->
            GraphEdge(
                sourceNodeIndex = sourceNodeIndex,
                targetNodeIndex = targetNodeIndex,
                value = state[sourceNodeIndex, targetNodeIndex]
            )
        }
    }

val edgeRecurrence = dataFrameOf(
    "sourceNodeIndex" to mappedToEdges.map { it.sourceNodeIndex },
    "targetNodeIndex" to mappedToEdges.map { it.targetNodeIndex },
    "numberOfEdges" to mappedToEdges.map { it.value },
)

In [ ]:
edgeRecurrence.plot {
    heatmap(
        x = sourceNodeIndex,
        y = targetNodeIndex,
    ) {
        fillColor(
            values = numberOfEdges.map { it == 0 }
        ) {
            scale = categorical(listOf(Color.BLACK,Color.WHITE))
            legend.name = "number of edges"
        }
    }
}

In [ ]:
edgeRecurrence.plot {
    numberOfEdges.distinct()
    bars {
        x(numberOfEdges.distinct())
        y(numberOfEdges.distinct().map { numberOfEdges.count { numberOfEdge -> it == numberOfEdge } })
    }
}

In [ ]:
val ratioOfExtinctEdges = edgeAccuranceMatrices
    .map { matrix ->
        Pair(
            matrix.indices[0].map { matrix[it].count { it == 0 } }.sum(),
            matrix.indices[0].map { matrix[it].count { it != 0 } }.sum(),
        )
    }

val extinctionTable = dataFrameOf(
    "index" to ratioOfExtinctEdges.indices.toList(),
    "extinct" to ratioOfExtinctEdges.map { it.first },
    "nonExtinct" to ratioOfExtinctEdges.map { it.second },
)

In [ ]:
extinctionTable
    .plot {
        layout {
            title = "Plot title"
            size = 3000 to 1000
        }
        x(index)
        line {
            y(extinct)
            color = Color.RED
        }
        line {
            y(nonExtinct)
            color = Color.BLUE
        }
    }

In [ ]:
val bestValuesData = mapStates(
    filePath = "D:/Research/TEMP/2024-03-01/2024-03-01T18/algorithmState.json",
    sampleDistance = 100,
) { 
    it.population.activesAsSequence()
        .minOf { it.cost?.get(0) ?: -1f } 
}

In [ ]:
val bestCostTable = dataFrameOf( 
    "indexes" to bestValuesData.indices.toList(),
    "bestCost" to bestValuesData,
)

In [ ]:
bestCostTable.plot { 
    line { 
        x(indexes)
        y(bestCost)
        color = Color.BLUE
    }
}

In [ ]:
val costsData  = mapStates(
    filePath = "D:/Research/TEMP/2024-03-04/2024-03-04T22/algorithmState.json",
    sampleDistance = 100
) {
    it.population.activesAsSequence()
        .map { it.cost?.get(0) ?: -1f }
        .toList()
        .toFloatArray()
}

In [ ]:
val costsTable = dataFrameOf(
    "index" to costsData.indices.toList(),
    "max" to costsData.map { it.max() },
    "avg" to costsData.map { it.average() },
    "min" to costsData.map { it.min() },
    "spread" to costsData
        .map { costs ->
            costs
                .map { abs(it - costs.average()) }
                .average()
        },
    "mean" to costsData
        .map { it.sorted()[it.size / 2] }
)

In [ ]:
costsTable.plot {
    x(index)
    line {
        y(max)
        color = Color.BLUE
    }
    line {
        y(min)
        color = Color.RED
    }
    line {
        y(avg)
        color = Color.PURPLE
    }
    line {
        y(spread)
        color = Color.GREEN
    }
    line {
        y(mean)
        color = Color.YELLOW
    }
}